### Fannying around with address forms

In [1]:
from i18naddress import normalize_address, InvalidAddress

In [2]:
address = normalize_address({
    'country_code': 'US',
    'country_area': 'California',
    'city': 'Mountain View',
    'postal_code': '94043',
    'street_address': '1600 Amphitheatre Pkwy'})

In [3]:
address

{'country_code': 'US',
 'country_area': 'CA',
 'city': 'MOUNTAIN VIEW',
 'postal_code': '94043',
 'street_address': '1600 Amphitheatre Pkwy',
 'city_area': '',
 'sorting_code': ''}

In [4]:
try:
    address = normalize_address({'country_code': 'US'})
except InvalidAddress as e:
    print(e.errors)

{'country_area': 'required', 'city': 'required', 'postal_code': 'required', 'street_address': 'required'}


In [4]:
%pdb

Automatic pdb calling has been turned ON


In [1]:
from saleor.account import i18n

In [34]:
d = {'csrfmiddlewaretoken': ['cxHqQBwfi6Dt6hhXpyHEdsoWdAY5AxMk7lmMFLYQiWcYpQOotkhQEscuHb3HyIda', 'cxHqQBwfi6Dt6hhXpyHEdsoWdAY5AxMk7lmMFLYQiWcYpQOotkhQEscuHb3HyIda'], 'address': ['new_address'], 'first_name': ['Christopher'], 'last_name': ['Kelly'], 'company_name': ['Nourish Balance Thrive'], 'street_address_1': ['471 Country Estates Ter'], 'street_address_2': [''], 'city': ['Santa Cruz'], 'country_area': ['CA'], 'postal_code': ['95060'], 'city_area': [''], 'country': ['US'], 'phone_1': [''], 'preview': [''], 'shipping_method': ['13'], 'cardholder-name': ['Christopher Kelly'], 'payment_method_id': ['pm_1FmLG5DGt0TthQJwmdlFHvWr']}

In [35]:
form = i18n.COUNTRY_FORMS['US'](d)

In [99]:
form = i18n.COUNTRY_FORMS['US']()

In [101]:
form

<AddressFormUS bound=False, valid=False, fields=(first_name;last_name;company_name;street_address_1;street_address_2;city;city_area;postal_code;country;country_area;phone)>

In [102]:
form.is_valid(), form.errors

(False, {})

In [103]:
form._errors

{}

In [6]:
[e.message for e in form.errors.as_data().get('country')]

['This field is required.']

In [21]:
i18naddress.__file__

'/home/cck197/saleor/venv2/lib/python3.7/site-packages/i18naddress/__init__.py'

In [104]:
i18n.__file__

'/home/cck197/saleor/saleor/account/i18n.py'

### Trying to update an existing `Order` using a new `Checkout`

In [76]:
from saleor.checkout.utils import *
from saleor.product.utils import *
from saleor.payment.utils import *
from saleor.payment.gateway import *

In [ ]:
pm_1Fr46MDGt0TthQJwfO6NYTw1
pm_1Fr46MDGt0TthQJwfO6NYTw1

In [133]:
# Stripe problem
d1 = {'amount': Decimal('16.50'), 'currency': 'USD', 'billing': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), 'shipping': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), 'order_id': 207, 'customer_ip_address': '10.0.2.2', 'customer_email': 'chris@nourishbalancethrive.com', 'token': 'pm_1FqqkdDGt0TthQJw0ISJSX9z', 'customer_id': 'cus_GKvtnmCUrDESc3', 'reuse_source': True}
d2 = {'amount': Decimal('5.00'), 'currency': 'USD', 'billing': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), 'shipping': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), 'order_id': '207_2', 'customer_ip_address': '10.0.2.2', 'customer_email': 'chris@nourishbalancethrive.com', 'token': 'pm_1FqqkdDGt0TthQJw0ISJSX9z', 'customer_id': 'cus_GKvtnmCUrDESc3', 'reuse_source': True}
d3 = {'amount': Decimal('16.50'), 'currency': 'USD', 'billing': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), 'shipping': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), 'order_id': 208, 'customer_ip_address': '10.0.2.2', 'customer_email': 'chris@nourishbalancethrive.com', 'token': 'pm_1FqqxYDGt0TthQJwP2RqWBVo', 'customer_id': 'cus_GKvtnmCUrDESc3', 'reuse_source': True}
d4 = {'amount': Decimal('5.00'), 'currency': 'USD', 'billing': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), 'shipping': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), 'order_id': '208_2', 'customer_ip_address': '10.0.2.2', 'customer_email': 'chris@nourishbalancethrive.com', 'token': 'pm_1FqqxYDGt0TthQJwP2RqWBVo', 'custom$r_id': 'cus_GKvtnmCUrDESc3', 'reuse_source': True}

In [136]:
d1

{'amount': Decimal('16.50'),
 'currency': 'USD',
 'billing': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'),
 'shipping': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'),
 'order_id': 207,
 'customer_ip_address': '10.0.2.2',
 'customer_email': 'chris@nourishbalancethrive.com',
 'token': 'pm_1FqqkdDGt0TthQJw0ISJSX9z',
 'customer_id': 'cus_GKvtnmCUrDESc3',
 'reuse_source': True}

In [137]:
d3

{'amount': Decimal('16.50'),
 'currency': 'USD',
 'billing': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'),
 'shipping': AddressData(first_name='Christopher', last_name='Kelly', company_name='', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'),
 'order_id': 208,
 'customer_ip_address': '10.0.2.2',
 'customer_email': 'chris@nourishbalancethrive.com',
 'token': 'pm_1FqqxYDGt0TthQJwP2RqWBVo',
 'customer_id': 'cus_GKvtnmCUrDESc3',
 'reuse_source': True}

In [150]:
user = User.objects.get(email="chris@nourishbalancethrive.com")

In [151]:
user.private_meta

{'payment-gateways': {'DUMMY': {'customer_id': None},
  'STRIPE': {'customer_id': 'cus_GNciwmWqsAiSWp'},
  'BRAINTREE': {'customer_id': '254012641'}}}

In [160]:
import stripe
stripe.api_key = 'sk_test_ZPbcFgfw0fJwDT5S3iTbKtvZ00PKAqC1JC'

In [161]:
customer_id = fetch_customer_id(user, "Stripe")

In [162]:
stripe.Customer.retrieve(customer_id)

INFO stripe message='Request to Stripe api' method=get path=https://api.stripe.com/v1/customers/cus_GNciwmWqsAiSWp [PID:26089:MainThread]
INFO stripe message='Stripe API response' path=https://api.stripe.com/v1/customers/cus_GNciwmWqsAiSWp response_code=200 [PID:26089:MainThread]


<Customer customer id=cus_GNciwmWqsAiSWp at 0x7fa91ae0bf50> JSON: {
  "address": null,
  "balance": 0,
  "created": 1576632998,
  "currency": null,
  "default_source": null,
  "delinquent": false,
  "description": null,
  "discount": null,
  "email": "chris@nourishbalancethrive.com",
  "id": "cus_GNciwmWqsAiSWp",
  "invoice_prefix": "DAA454C8",
  "invoice_settings": {
    "custom_fields": null,
    "default_payment_method": null,
    "footer": null
  },
  "livemode": false,
  "metadata": {},
  "name": null,
  "object": "customer",
  "phone": null,
  "preferred_locales": [],
  "shipping": null,
  "sources": {
    "data": [],
    "has_more": false,
    "object": "list",
    "total_count": 0,
    "url": "/v1/customers/cus_GNciwmWqsAiSWp/sources"
  },
  "subscriptions": {
    "data": [],
    "has_more": false,
    "object": "list",
    "total_count": 0,
    "url": "/v1/customers/cus_GNciwmWqsAiSWp/subscriptions"
  },
  "tax_exempt": "none",
  "tax_ids": {
    "data": [],
    "has_more": fa

In [164]:
payment_method = stripe.PaymentMethod.retrieve(
  "pm_1Fr51ODGt0TthQJw4hzissd3",
)

INFO stripe message='Request to Stripe api' method=get path=https://api.stripe.com/v1/payment_methods/pm_1Fr51ODGt0TthQJw4hzissd3 [PID:26089:MainThread]
INFO stripe message='Stripe API response' path=https://api.stripe.com/v1/payment_methods/pm_1Fr51ODGt0TthQJw4hzissd3 response_code=200 [PID:26089:MainThread]


In [166]:
payment_method.customer

'cus_GNciwmWqsAiSWp'

In [143]:
store_customer_id(user, "Stripe", None)

In [138]:
user

<User: chris@nourishbalancethrive.com>

In [197]:
checkout = Checkout.objects.first()

In [148]:
checkout

Checkout(quantity=1)

In [192]:
d = {'csrfmiddlewaretoken': ['XbXDgFkLm1qi3NVD7m5fFNAnyW2yBQShyuuHHvRRUimVpD93QND1rzutPlrT3hgX', 'XbXDgFkLm1qi3NVD7m5fFNAnyW2yBQShyuuHHvRRUimVpD93QND1rzutPlrT3hgX', 'XbXDgFkLm1qi3NVD7m5fFNAnyW2yBQShyuuHHvRRUimVpD93QND1rzutPlrT3hgX'], 'first_name': ['Christopher'], 'last_name': ['Kelly'], 'email': ['chris@nourishbalancethrive.com'], 'country': ['US'], 'street_address_1': ['471 Country Estates Ter'], 'street_address_2': [''], 'city': ['Santa Cruz'], 'country_area': ['CA'], 'postal_code': ['95060'], 'city_area': [''], 'preview': [''], 'cardholder-name': ['Christopher Kelly'], 'payment_method_id': [''], 'payment_method_nonce': ['80005b7d-104f-0c4f-6bae-d6fe85ca46dd'], 'amount': ['19.00']}

In [193]:
from saleor.payment.gateways.stripe.forms import StripePaymentForm
#from saleor.payment.gateways.braintree.forms import BraintreePaymentForm

In [194]:
form = StripePaymentForm(data=d, payment_information=None)

In [196]:
form.is_valid()

True

In [186]:
form.get_payment_token()

AttributeError: 'StripePaymentForm' object has no attribute 'cleaned_data'

In [110]:
get_valid_shipping_methods_for_checkout(checkout, [])

In [111]:
get_valid_shipping_methods_for_checkout??

In [78]:
import inspect
inspect.getfile(Address.__class__)

'/home/cck197/saleor/venv2/lib/python3.7/site-packages/django/db/models/base.py'

In [79]:
address = Address(country="US")

In [80]:
address.first_name

''

In [81]:
address.country

Country(code='US')

In [51]:
checkout.shipping_address

<Address: Nourish Balance Thrive - Christopher Kelly>

In [202]:
checkout = Checkout.objects.first()

In [203]:
checkout.delete()

(2,
 {'checkout.Checkout_gift_cards': 0,
  'checkout.CheckoutLine': 1,
  'checkout.Checkout': 1})

In [199]:
Foo = 1

In [201]:
Foo

1

In [19]:
checkout.get_meta('funnel', 'funnel')

{'slug': 'juice-funnel', 'token': None, 'funnel_index': 1}

In [214]:
o1 = Order.objects.first(); o1.token, o1

('9a7c7b13-d8e2-4d2c-ab45-62845b19e335', <Order #221>)

In [215]:
o1.payments.all()

<QuerySet [Payment(gateway=Braintree, is_active=True, created=2019-12-18 21:05:46.423449+00:00, charge_status=fully-charged), Payment(gateway=Braintree, is_active=True, created=2019-12-18 21:06:39.913333+00:00, charge_status=fully-charged)]>

In [115]:
o1.delete()

ProtectedError: ("Cannot delete some instances of model 'Order' because they are referenced through a protected foreign key: 'Payment.order'", <QuerySet [Payment(gateway=Stripe, is_active=True, created=2019-12-18 00:22:09.531980+00:00, charge_status=not-charged)]>)

In [87]:
o1.id = '179_1'

In [ ]:
o1.

In [88]:
o1.save()

IntegrityError: duplicate key value violates unique constraint "order_order_token_key"
DETAIL:  Key (token)=(9fc8d000-2b54-4838-bae8-36cc9302d82d) already exists.


In [84]:
o1.user.private_meta

{'payment-gateways': {'BRAINTREE': {'customer_id': '254012641'}}}

In [69]:
o1.is_fully_paid()

False

In [70]:
o1.billing_address

In [71]:
o1.shipping_address.as_data()

AttributeError: 'NoneType' object has no attribute 'as_data'

In [49]:
o1.shipping_method

In [32]:
d

{'csrfmiddlewaretoken': ['cxHqQBwfi6Dt6hhXpyHEdsoWdAY5AxMk7lmMFLYQiWcYpQOotkhQEscuHb3HyIda',
  'cxHqQBwfi6Dt6hhXpyHEdsoWdAY5AxMk7lmMFLYQiWcYpQOotkhQEscuHb3HyIda'],
 'address': ['new_address'],
 'first_name': ['Christopher'],
 'last_name': ['Kelly'],
 'company_name': ['Nourish Balance Thrive'],
 'street_address_1': ['471 Country Estates Ter'],
 'street_address_2': [''],
 'city': ['Santa Cruz'],
 'country_area': ['CA'],
 'postal_code': ['95060'],
 'city_area': [''],
 'country': ['US'],
 'phone_1': [''],
 'preview': [''],
 'shipping_method': ['13'],
 'cardholder-name': ['Christopher Kelly'],
 'payment_method_id': ['pm_1FmLG5DGt0TthQJwmdlFHvWr']}

In [29]:
gettataaao1.redirect

AttributeError: 'Order' object has no attribute 'redirect'

In [22]:
o1.billing_address

In [72]:
o1.delete()

(3,
 {'order.Order_gift_cards': 0,
  'product.DigitalContentUrl': 0,
  'order.FulfillmentLine': 0,
  'order.OrderEvent': 1,
  'order.OrderLine': 1,
  'order.Order': 1})

In [26]:
user = User.objects.get(email='chris@nourishbalancethrive.com')

In [27]:
user.addresses.all()

<AddressQueryset [<Address: Nourish Balance Thrive - Christopher Kelly>]>

In [17]:
[addr.delete() for addr in user.addresses.all()]

[(2, {'account.User_addresses': 1, 'account.Address': 1}),
 (2, {'account.User_addresses': 1, 'account.Address': 1}),
 (2, {'account.User_addresses': 1, 'account.Address': 1}),
 (2, {'account.User_addresses': 1, 'account.Address': 1})]

In [90]:
o1.currency

'USD'

In [98]:
print(o1.total_gross.amount)

41.50


In [17]:
o1.token

'374ca1c7-5aa8-4b44-9d9c-f1147f818e33'

In [15]:
upsell_order(o1, checkout, "")

<Order #111>

In [16]:
o1.payments.first().gateway

'Braintree'

In [17]:
len(o1.payments.all())

1

In [16]:
%pdb

Automatic pdb calling has been turned OFF


In [18]:
payment = o1.payments.first()
extra_data = {f"upsell_payment_number": f'{len(o1.payments.all())}'}
payment_ = create_payment(
      gateway=payment.gateway,
      currency=o1.total.gross.currency,
      email=o1.user_email,
      billing_address=o1.billing_address,
      customer_ip_address=payment.customer_ip_address,
      total=abs(o1.total_balance.amount),
      order=o1,
      extra_data=extra_data,
  )

In [19]:
extra_data

{'upsell_payment_number': '1'}

In [20]:
payment_, payment_.pk

(Payment(gateway=Braintree, is_active=True, created=2019-11-22 21:12:21.847482+00:00, charge_status=not-charged),
 150)

In [21]:
transaction = payment.transactions.first()
token = transaction.gateway_response.get('payment_method', '')
customer_id = transaction.customer_id

In [23]:
customer_id, token

('896095806', '')

In [24]:
transaction = process_payment(payment_, token, store_source=True, customer_id=customer_id)

process_payment: token:  store_source: True
process_payment: reuse_source: True
braintree.authorize: payment_information: PaymentData(amount=Decimal('7.00'), currency='USD', billing=AddressData(first_name='Julia', last_name='Kelly', company_name='Nourish Balance Thrive', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), shipping=AddressData(first_name='Julia', last_name='Kelly', company_name='Nourish Balance Thrive', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), order_id=111, customer_ip_address='10.0.2.2', customer_email='chris@nourishbalancethrive.com', token='', customer_id='896095806', reuse_source=True)
authorize: gateway_response: {'transaction_id': 'ah26wx02', 'currency': 'USD', 'amount': Decimal('7.00'), 'credit_card': {'token':

PaymentError: Transaction was unsuccessful

In [ ]:
GatewayResponse(is_success=True, action_required=False, kind='capture', amount=Decimal('13.90'), currency='USD', transaction_id='4j7vnf6v', error='', cust
omer_id='409815689', card_info=None, raw_response={'transaction_id': '4j7vnf6v', 'currency': 'USD', 'amount': Decimal('13.90'), 'credit_card': {'token': '43thd9', 'bin': None, 'last
_4': None, 'card_type': None, 'expiration_month': '', 'expiration_year': '', 'customer_location': None, 'cardholder_name': None, 'image_url': 'https://assets.braintreegateway.com/pa
yment_method_logo/unknown.png?environment=sandbox', 'prepaid': 'Unknown', 'healthcare': 'Unknown', 'debit': 'Unknown', 'durbin_regulated': 'Unknown', 'commercial': 'Unknown', 'payro
ll': 'Unknown', 'issuing_bank': 'Unknown', 'country_of_issuance': 'Unknown', 'product_id': 'Unknown', 'global_id': 'cGF5bWVudG1ldGhvZF9jY180M3RoZDk', 'account_type': None, 'unique_n
umber_identifier': None, 'venmo_sdk': False}, 'customer_id': '409815689', 'errors': []}), error: None

In [17]:
transaction.pk

102

In [214]:
19+10.9

29.9

In [217]:
o1.save()

In [175]:
o1.payments.all()

<QuerySet [Payment(gateway=Dummy, is_active=True, created=2019-11-14 15:01:58.499543+00:00, charge_status=fully-charged), Payment(gateway=Dummy, is_active=True, created=2019-11-14 17:05:32.774685+00:00, charge_status=fully-charged)]>

In [144]:
payment2.transactions.all()

<QuerySet [Transaction(type=capture, is_success=True, created=2019-11-13 16:45:48.503291+00:00)]>

In [152]:
o1.payments.all()

<QuerySet [Payment(gateway=Dummy, is_active=True, created=2019-11-13 16:42:32.471117+00:00, charge_status=fully-charged), Payment(gateway=Dummy, is_active=True, created=2019-11-13 16:45:38.533943+00:00, charge_status=fully-charged)]>

In [145]:
payment.transactions.add(transaction)

In [146]:
payment.save()

In [147]:
payment.total

Decimal('13.90')

In [150]:
[t.amount for t in payment.transactions.all()]

[Decimal('13.90'), Decimal('-7.00'), Decimal('7.00'), Decimal('0.00')]

I think the code above works, but the methods in `Order` and maybe `Payment` don't work for multiple payments. Add new methods and adjust dashboard templates accordingly.